In [64]:
import folium
import pandas as pd
import numpy as np
from statistics import mean
from Distance import DistanceMatrix
from ortools.constraint_solver import routing_enums_pb2 
from ortools.constraint_solver import pywrapcp 

# Test data

In [65]:
df = pd.read_csv('./Data/Test.csv')
df

,name,lat,lon,TimeWindow1,TimeWindow2,ServiceTime,SO,Weight,Volume
0,1,15.6224,100.1145,"(1, 1439)","(1, 1439)",20,Z,0,0
1,2,15.7000,100.1400,"(420, 1080)","(480, 1080)",15,D240806510549,20000,22
2,3,16.4734,99.5337,"(420, 1080)","(480, 1080)",15,D240806510044,15312,39
3,4,16.4734,99.5337,"(420, 1080)","(480, 1080)",15,D240806510043,421,5
4,5,15.7471,99.7920,"(420, 1080)","(480, 1080)",15,D240806510035,984,12
...,...,...,...,...,...,...,...,...,...
222,223,16.4839,99.5203,"(420, 1080)","(420, 1080)",15,C1350922500,343000,1655
223,224,16.4839,99.5203,"(420, 1080)","(420, 1080)",15,C1350922499,650000,3288
224,225,16.4839,99.5203,"(420, 1080)","(420, 1080)",15,C1350922497,770000,3632
225,226,16.4839,99.5203,"(420, 1080)","(420, 1080)",15,C1350922496,16860,62


In [66]:
demand = df["Volume"]
demands = demand.to_list()
print(demands)

[0, 22, 39, 5, 12, 3, 36, 16, 3, 37, 25, 37, 25, 18, 131, 3, 9, 819, 3, 10, 3, 296, 3, 3, 10, 3, 3, 15, 3, 155, 3, 3, 3, 59, 505, 293, 73, 5, 84, 9, 3, 13, 73, 3, 5, 18, 7, 3, 6, 3, 5, 14, 14, 3, 5, 33, 3, 13, 3, 3, 55, 41, 3, 5, 3, 3, 5, 5, 37, 3, 8, 37, 110, 11, 18, 3828, 3, 310, 55, 1, 1, 1, 1, 35, 1, 125, 35, 94, 365, 23, 13, 10, 32, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 70, 35, 55, 14, 14, 30, 297, 4, 383, 127, 304, 383, 299, 105, 190, 35, 163, 18, 30, 35, 14, 108, 78, 1, 1, 1, 21, 10, 10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1647, 78, 78, 29, 116, 4, 19, 71, 7, 124, 272, 36, 10, 5, 1, 1, 1, 1, 1, 413, 10, 140, 250, 63, 157, 231, 45, 38, 32, 32, 32, 38, 32, 14, 12, 22, 22, 22, 202, 22, 41, 49, 2400, 2400, 2400, 90, 295, 196, 77, 508, 0, 2, 0, 1, 29, 616, 3, 174, 1659, 140, 140, 70, 90, 6, 1684, 43, 66, 127, 625, 1655, 3288, 3632, 62, 55]


In [67]:
print(f"mean = {mean(demands)}")
print(f"max = {max(demands)}")
print(f"Sum = {sum(demands)}") 
#Vehicle capacity must be adequate to meet demands in order to find a solution.

mean = 168
max = 3828
Sum = 38136


In [68]:
print(len(demands))

227


In [69]:
split_index = int(len(demands) * 0.8)
demands_half = demands[:split_index]
print(f"split_index = {split_index}") 
print(f"demands_half = {sum(demands_half)}") 

split_index = 181
demands_half = 15031


In [70]:
dicttest = df.to_dict(orient='records')
distest = np.array(DistanceMatrix(dicttest))
distest


array([[  0,  11, 147, ..., 149, 149, 149],
       [ 11,   0, 140, ..., 142, 142, 142],
       [147, 140,   0, ...,   2,   2,   2],
       ...,
       [149, 142,   2, ...,   0,   0,   0],
       [149, 142,   2, ...,   0,   0,   0],
       [149, 142,   2, ...,   0,   0,   0]])

In [71]:
assert len(demands) == len(distest)

# Data location

In [72]:
#df = pd.read_csv('./Data/locations.csv')
#df

### Spilt data to test

In [73]:
df1 = df.iloc[:split_index]

In [74]:
dict1 = df1.to_dict(orient='records')
dis = DistanceMatrix(df.to_dict(orient='records'))
dis1 = DistanceMatrix(dict1)
print(np.array(dis1))

[[  0  11 147 ...  71  12 113]
 [ 11   0 140 ...  65   0 124]
 [147 140   0 ...  76 139 200]
 ...
 [ 71  65  76 ...   0  64 139]
 [ 12   0 139 ...  64   0 124]
 [113 124 200 ... 139 124   0]]


## First run

In [75]:
def create_data_model():
    data = {}
    data["distance_matrix"] = dis1
    data["demands"] = demands_half #[0,2,3,1,3]
    data["vehicle_capacities"] = [10000,10000,10000,10000,10000,10000,10000,10000,10000,10000]
    data["num_vehicles"] = 10
    data["depot"] = 0
    return data

In [76]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)}"
    print(dropped_nodes)
    # Display routes
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")

def get_routes(solution, routing, manager):
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    """ penalty = 1000
    for node in range(1, len(data["distance_matrix"])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty) """

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        
        routes = get_routes(solution, routing, manager)
        
        for i, route in enumerate(routes):
            print('Route', i, route)  

        return routes
    
    else:
        print("No solution founded!")


if __name__ == "__main__":
    routes = main()

Objective: 634
Dropped nodes:
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 4:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 5:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 6:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 7:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 8:
 0 Load(0) ->  122 Load(127) ->  174 Load(267) ->  125 Load(566) ->  124 Load(949) ->  120 Load(953) ->  173 Load(963) ->  34 Load(1468) ->  114 Load(1503) ->  28 Load(1506) ->  156 Load(1535) ->  

In [77]:
print(routes)

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 122, 174, 125, 124, 120, 173, 34, 114, 28, 156, 155, 13, 16, 17, 19, 20, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 90, 113, 128, 133, 134, 177, 176, 180, 106, 59, 55, 54, 50, 44, 42, 40, 39, 38, 144, 141, 4, 171, 178, 89, 88, 115, 21, 9, 36, 35, 15, 14, 12, 18, 2, 3, 5, 6, 7, 8, 10, 11, 119, 166, 154, 153, 91, 78, 123, 121, 158, 77, 79, 84, 95, 98, 101, 146, 147, 148, 149, 151, 105, 0], [0, 118, 117, 116, 165, 136, 137, 138, 145, 150, 152, 161, 162, 163, 164, 167, 168, 169, 170, 172, 175, 179, 135, 159, 143, 142, 140, 139, 132, 130, 129, 126, 112, 111, 110, 109, 108, 107, 104, 103, 102, 100, 99, 97, 96, 94, 93, 92, 87, 86, 85, 82, 81, 80, 1, 160, 75, 157, 76, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 58, 57, 56, 53, 52, 51, 49, 48, 47, 46, 45, 43, 41, 37, 131, 83, 127, 0]]


## Plot first run

In [78]:
def mapping(df,routes):
    # Remove unused routes [0,0]
    filtered_routes = [route for route in routes if len(route) != 2]
    # Map coordinates for the remaining routes
    routes_coordinates = {}
    for i, route in enumerate(filtered_routes):
        coordinates = []
        for path in route:
            coordinates.append([df.iloc[path]['lat'], df.iloc[path]['lon']])
        routes_coordinates[i] = coordinates
    
    return routes_coordinates

coordinates = mapping(df,routes)

map_center = [df['lat'].mean(), df['lon'].mean()]
custom ="cartodb positron"
map = folium.Map(location=map_center, zoom_start=4,tiles=custom)

# Choose location
locations = df

# Name of map
map_name = "./maps/CVRP_fristrun.html"
map.save(map_name)

# Add marker
dest = df.drop(index=0)
destination = dest.iloc[:split_index]

for _,row in destination.iterrows():
    folium.Circle(
        #location=[row['lat'],row['lon']],
        location=[row['lat'],row['lon']],
        radius=2,  # Radius in pixels
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['name'],
    ).add_to(map)

""" folium.Marker(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    popup=df_locations.iloc[0]['name'],
    icon=folium.Icon(color='lightred') 
    ).add_to(map) """

folium.Circle(
    location=[df.iloc[0]['lat'],df.iloc[0]['lon']],
    radius=2,  # Radius in pixels
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    tooltip=df.iloc[0]['name']
).add_to(map)

map.save(map_name)

colors = ["red", "yellow", "green", "cyan", "blue", "magenta"]

# Add edge
for i in range(len(coordinates)):
    folium.PolyLine(
        locations=coordinates[i],
        color=colors[i],
        weight=1,
        tooltip="TSP"
    ).add_to(map)
map.save(map_name)

## Second Run

### Fix Node with Vehicle

- **Objective:** Ensure that specific nodes are assigned to the correct vehicles in the routing problem.

1. **Identify Nodes and Vehicles:**
   - Determine which nodes need to be serviced by specific vehicles `use answer (routes) form frist run`.
 
2. **Update Routing Model:**
   - Use the `SetAllowedVehiclesForIndex` method to restrict which vehicles can visit certain nodes based on the initial routing data or constraints.

3. **Code:**

   ```python
   for vehicle_id, route in data["initialRoute"].items():
       for node in route:
           node_index = manager.NodeToIndex(node)
           routing.SetAllowedVehiclesForIndex([vehicle_id], node_index)


In [79]:
def fixed_node(routes):
    initialRoute = {}
    for index, route in enumerate(routes):
        nodes = []
        for node in route:
            if node != 0:
                nodes.append(node)
        initialRoute[index] = nodes

    return initialRoute

In [80]:
def create_data_model():
    data = {}
    data["distance_matrix"] = dis
    data["demands"] = demands
    data["vehicle_capacities"] = [10000,10000,10000,10000,10000,10000,10000,10000,10000,10000]
    data["num_vehicles"] = 10
    data["depot"] = 0
    data["initialRoute"] = fixed_node(routes)
    return data

In [81]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)}"
    print(dropped_nodes)
    # Display routes
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")

def get_routes(solution, routing, manager):
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )
    
    """ routing.SetAllowedVehiclesForIndex([0,1,4,0],0) """
    """ routing.SetAllowedVehiclesForIndex([0,2,3,0],1) """

    for vehicle_id, route in data["initialRoute"].items():
        for node in route:
            node_index = manager.NodeToIndex(node)
            routing.SetAllowedVehiclesForIndex([vehicle_id], node_index)
    
    # Set allowed vehicles for each node based on the initial route.
    #for v in initialRoute:
        #route = initialRoute[v]
            #for node in route:
                #node_index = manager.NodeToIndex(node)
                #routing.SetAllowedVehiclesForIndex([v], node_index)

    penalty = 1000
    for node in range(1, len(data["distance_matrix"])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

        routes = get_routes(solution, routing, manager)

        for i, route in enumerate(routes):
            print('Route', i, route)

        return routes,solution.ObjectiveValue(),total_distance, total_load
    
    else:
        print("No solution founded!")

if __name__ == "__main__":
    routes, solution, total_distance, total_load = main()

Objective: 6331
Dropped nodes: 77 121 123 153 178
Objective: 6331
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 4:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 5:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 6:
 0 Load(0) ->  216 Load(6) ->  201 Load(83) ->  210 Load(257) ->  198 Load(347) ->  212 Load(487) ->  214 Load(557) ->  223 Load(3845) ->  224 Load(7477) ->  225 Load(7539) ->  226 Load(7594) ->  211 Load(9253) ->  0 Load(9253)
Distance of the route: 298m
Load of the route: 9253

Route for vehicle 7:
 0 Load(0) ->  209 Load(3) ->  202 Load(511) ->

NameError: name 'total_distance' is not defined

## Plot Second run

In [ ]:
def mapping(df,routes):
    # Remove unused routes [0,0]
    filtered_routes = [route for route in routes if len(route) != 2]
    # Map coordinates for the remaining routes
    routes_coordinates = {}
    for i, route in enumerate(filtered_routes):
        coordinates = []
        for path in route:
            coordinates.append([df.iloc[path]['lat'], df.iloc[path]['lon']])
        routes_coordinates[i] = coordinates
    
    return routes_coordinates

coordinates = mapping(df,routes)

map_center = [df['lat'].mean(), df['lon'].mean()]
custom ="cartodb positron"
map = folium.Map(location=map_center, zoom_start=4, tiles=custom)

# Choose location
locations = df

# Name of map
map_name = "./maps/CVRP_secondrun.html"
map.save(map_name)

# Add marker
destination = df.drop(index=0)

for _,row in destination.iterrows():
    folium.Circle(
        location=[row['lat'],row['lon']],
        radius=2,  # Radius in pixels
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['name'],
    ).add_to(map)

""" folium.Marker(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    popup=df_locations.iloc[0]['name'],
    icon=folium.Icon(color='lightred') 
    ).add_to(map) """

folium.Circle(
    location=[df.iloc[0]['lat'],df.iloc[0]['lon']],
    radius=2,  # Radius in pixels
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    tooltip=df.iloc[0]['name']
).add_to(map)

map.save(map_name)

colors = ["red", "yellow", "green", "cyan", "blue", "magenta"]

# Add edge
for i in range(len(coordinates)):
    folium.PolyLine(
        locations=coordinates[i],
        color=colors[i],
        weight=1,
        tooltip="TSP"
    ).add_to(map)

""" folium.PolyLine(
    locations=coordinates[0],
    color="#ff6f00",
    weight=0.25,
    tooltip="TSP"
).add_to(map)

folium.PolyLine(
    locations=coordinates[1],
    color="#5B99C2",
    weight=0.5,
    tooltip="TSP"
).add_to(map) """

map.save(map_name)

# Calculate

In [ ]:
solution

6331